<a href="https://colab.research.google.com/github/liisareet/working-files/blob/main/Python_project_RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Workbook setup

import pandas as pd
import pandas_gbq
import numpy as np
import plotly.express as px
print('Setup complete!')

Setup complete!


In [4]:
# Authenticate user

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [19]:
# Connect to BigQuery
from google.cloud import bigquery
client = bigquery.Client()
project_id = 'tc-da-1'

"""
RFM analysis based on customer dataset, 1 year of data - 2010-12-01 to 2011-12-01

Segments are based on separate r_, f_ and m_scores , rather than a combined RFM score
"""

In [20]:
# Create Bigquery job object
sql = """
WITH
-- Clean dataset
customerdata AS (
SELECT
CustomerID, InvoiceDate, InvoiceNo, SUM(LinePrice) AS TotalDue
FROM (
SELECT CustomerID, InvoiceDate, InvoiceNo, (Quantity*UnitPrice) AS LinePrice
FROM `tc-da-1.turing_data_analytics.rfm`
WHERE CustomerId IS NOT NULL
AND InvoiceDate BETWEEN '2010-12-01' AND '2011-12-01'
AND (Quantity*UnitPrice) > 0)
GROUP BY CustomerID, InvoiceDate, InvoiceNo),

-- Calculation for F & M
frequency_monetary AS (
  SELECT CustomerID, MAX(InvoiceDate) AS last_purchase_date,
  COUNT(DISTINCT InvoiceNo) AS frequency,
  SUM(TotalDue) AS monetary
  FROM customerdata
  GROUP BY CustomerID
),

-- Calculation for R
recency_t AS (
SELECT *,
DATE_DIFF(reference_date,last_purchase_date,DAY) AS recency
FROM (SELECT *,MAX(last_purchase_date) OVER () + INTERVAL 1 DAY AS reference_date FROM frequency_monetary)),

-- Determine quartiles for RFM
quartiles AS (
SELECT  a.*,
    b.percentiles[offset(25)] AS m25,
    b.percentiles[offset(50)] AS m50,
    b.percentiles[offset(75)] AS m75,
    c.percentiles[offset(25)] AS f25,
    c.percentiles[offset(50)] AS f50,
    c.percentiles[offset(75)] AS f75,
    d.percentiles[offset(25)] AS r25,
    d.percentiles[offset(50)] AS r50,
    d.percentiles[offset(75)] AS r75
FROM
    recency_t a,
    (SELECT APPROX_QUANTILES(monetary, 100) percentiles FROM
    recency_t) b,
    (SELECT APPROX_QUANTILES(frequency, 100) percentiles FROM
    recency_t) c,
    (SELECT APPROX_QUANTILES(recency, 100) percentiles FROM
    recency_t) d),

--Assign RFM Score
scoring AS (
SELECT *,
-- recency scoring is reversed
CASE WHEN recency <= r25 THEN 4
WHEN recency <= r50 AND recency > r25 THEN 3
WHEN recency <= r75 AND recency > r50 THEN 2
WHEN recency > r75 THEN 1
END AS r_score,
CASE WHEN frequency <= f25 THEN 1
WHEN frequency <= f50 AND frequency > f25 THEN 2
WHEN frequency <= f75 AND frequency > f50 THEN 3
WHEN frequency > f75 THEN 4
END AS f_score,
CASE WHEN monetary <= m25 THEN 1
WHEN monetary <= m50 AND monetary > m25 THEN 2
WHEN monetary <= m75 AND monetary > m50 THEN 3
WHEN monetary > m75 THEN 4
END AS m_score
FROM quartiles)

SELECT
CustomerID,
recency,
monetary,
frequency,
r_score,
f_score,
m_score,
CASE
WHEN (r_score = 4 AND f_score = 4 AND m_score = 4) THEN 'Best Customers'
WHEN (r_score = 4 AND f_score IN(3,4) AND m_score IN (3,4)) THEN 'Loyal Customers'
WHEN (r_score IN (3,4) AND f_score > 1 AND m_score IN (2,3,4)) THEN 'Potential Loyalists'
WHEN (r_score = 4 AND f_score = 1 AND m_score IN (1,2,3,4)) THEN 'New customers'
WHEN (r_score = 4 AND f_score IN (1,2,3) AND m_score IN (1,2,3,4)) THEN 'Recent customers'
WHEN (r_score IN (2,3,4) and f_score IN (3,4) AND m_score = 1) THEN 'Price sensitive'
WHEN (r_score = 2 AND f_score IN (3,4) AND m_score IN (3,4)) THEN 'Needing attention'
WHEN (r_score IN (2,3) AND f_score IN (1,2) AND m_score IN (1,2,3,4)) THEN 'About to sleep'
WHEN (r_score = 1 AND f_score IN (2,3) AND m_score IN (2,3)) THEN 'Slipping customers'
WHEN (r_score = 1 AND f_score = 4 AND m_score = 4) THEN 'At risk'
WHEN (r_score = 1 AND f_score = 1 AND m_score IN (1,2,3,4)) THEN 'Lost'
    ELSE 'Other' -- Covers all remaining combinations
  END AS customer_segment
FROM scoring
"""

In [26]:
# Execute the query, create dataframe and check result
df = pandas_gbq.read_gbq(sql, project_id=project_id)
print(df.head())
print('no of rows in dataframe: ', df.shape[0])

Downloading: 100%|██████████|
   CustomerID  recency  monetary  frequency  r_score  f_score  m_score  \
0       16900        8    887.11          3        4        3        3   
1       13067       74    115.46          1        2        1        1   
2       17932       20   1331.57          3        3        3        3   
3       12574      307    218.45          1        1        1        1   
4       13858       49    216.17          1        3        1        1   

      customer_segment  
0      Loyal Customers  
1       About to sleep  
2  Potential Loyalists  
3                 Lost  
4       About to sleep  
no of rows in dataframe:  4297


In [ ]:
"""
Segmentation overview:

Best customer           Bought recently and often and spend the most
Loyal customers         Spend good money with us often. Responsive to promotions
Potential loyalists     Recent customers, spent a good amount and bought more than once
New customers           Bought most recently and first order
Recent Customers        Bought recently, but haven’t spent much
Price sensitive         Have frequency score above 2, but have very low spend
Needing attention       Above average recency, frequency and monetary values, but may not have bought very recently
About to sleep          Below average recency, frequency and monetary values. Will lose if not reactivated
Slipping customers      Spent big money and purchased often, but haven’t returned in a long time. Need to bring them back
Can’t lose them         Made biggest purchases and often, but haven’t returned in a long time
Hibernating             Last purchase was long back. Low spenders, low number of orders
Lost                    Lowest recency, frequency and monetary scores
"""

In [36]:
# Overview of dataset
customer_count = df['CustomerID'].nunique()
total_sales = df['monetary'].sum()
average_order_value = (total_sales / df['frequency'].sum()).round(2)
average_frequency = df['frequency'].mean().round()

print(f"Count of Customers: {customer_count}")
print(f"Total Sales: {total_sales} USD")
print(f"Average Order Value: {average_order_value} USD")
print(f"Average Frequency: {average_frequency}")

Count of Customers: 4297
Total Sales: 8393215.114 USD
Average Order Value: 472.75 USD
Average Frequency: 4.0


In [55]:
# Create bubble chart to show customer segment sizes

agg_df = df.groupby('customer_segment').agg(
    recency_avg=('recency', 'mean'),
    frequency_avg=('frequency', 'mean'),
    monetary_total=('monetary', 'sum')
).reset_index()

fig = px.scatter(
    agg_df,
    x='recency_avg',
    y='frequency_avg',
    size='monetary_total',
    color='customer_segment',
    text='customer_segment',
    hover_name='customer_segment',
    title='Customer Segment Sizes (bubble size indicating monetary value)',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    labels={'recency_avg': 'Average Recency(days)', 'frequency_avg': 'Average Frequency', 'monetary_total': 'Total Monetary',},
    size_max=50
)

fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False, width=1200, height=700)
fig.show()

In [65]:
# Create the treemap
customer_segment_counts = df['customer_segment'].value_counts().reset_index()
customer_segment_counts.columns = ['customer_segment', 'count']

# Create the treemap
fig = px.treemap(customer_segment_counts,
                 path=['customer_segment'],
                 values='count',
                 title='Customer Segmentation Split by count of customers',
                 color='customer_segment',
                 color_discrete_sequence=px.colors.qualitative.Prism)

# Customize the treemap to show the segment name and percentage of total
fig.data[0].textinfo = 'label+percent root'
fig.update_layout(width=1700, height=700)
fig.show()

In [66]:
segment_monetary_values = df.groupby('customer_segment')['monetary'].sum().reset_index()

# Calculate the percentage of total for each segment
segment_monetary_values['percentage_of_total'] = (segment_monetary_values['monetary'] / total_sales) * 100

# Create the bar chart
fig = px.bar(segment_monetary_values,
             x='customer_segment',
             y='monetary',
             text='percentage_of_total',
             title='Monetary Value of Customer Segments',
             color='customer_segment',
             color_discrete_sequence=px.colors.qualitative.Prism)

# Format the annotations to show as percentages with one decimal
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')

# Update layout for clearer visualization
fig.update_layout(xaxis_title='Customer Segment',
                  yaxis_title='Monetary Value',
                  uniformtext_minsize=8,
                  uniformtext_mode='hide',
                  showlegend=False,
                  width=1700, height=700)

fig.show()

In [ ]:
"""
Insights and suggestions:

Even though best customers make up only 10% of the customer base during this time period, their combined monetary value from total sales is nearly half of total sales.
The second segment with the biggest monetary values is also one of the largest, when it comes to the number of customers - Potential loyalists

When focusing marketing efforts and budget, it's important to keep in mind that it's always cheaper to keep existing customers than to gain new ones.
I would mainly focus the efforts on the Best Customers and Potential Loyalists segments.
It's important to keep up engagement with them and make sure they also know you appreciate their loyalty. You could offer loyalty programmes or the opportunity to test new products before everyone else.

As a secondary priority I would try and reengage with the needing attention and about to sleep segments, but with a lower budget - for example recommending popular products through email-campaigns.

"""